In [1]:
import pandas as pd
import numpy as np
import string
import pickle
import re

import spacy
from spacy.lang.id import Indonesian

import nltk
from nltk.corpus import wordnet as wn

import gensim
from gensim import corpora
import pyLDAvis.gensim

nltk.download('wordnet')
nlp = Indonesian()  # use directly
stopwords = spacy.lang.id.stop_words.STOP_WORDS
exclude = set(string.punctuation)

C:\Users\USER\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\USER\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
C:\Users\USER\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\USER\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
C:\Users\USER\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
data = pd.read_csv("dataset.csv")
label_drop = ["web-scraper-order","web-scraper-start-url","pengadu"]
data = data.drop(columns=label_drop)
data = data.replace({'\n':' '},regex=True)
data = data.replace({' +':' '},regex=True)
data.tail()

,diposisi,judul,keluhan,tipe
12856,Kementerian Sosial,Orang-Orang Mampu Mendapatkan BLSM,KPS di daerah desa sambong belum ada BLSM..dan...,Bantuan Langsung Sementara Masyarakat (BLSM)
12857,Kantor Walikota Jakarta Timur,Pungli di Kecamatan Makasar Dalam Pengurusan IMB,Hal melaporkan Staf Perizinan Bangunan Kec. Ma...,Reformasi Birokrasi dan Tata Kelola
12858,Kantor Walikota Jakarta Utara,Kader-kader Kelurahan Cilincing Memanfaatkan P...,Yth. Pemerintah Provinsi DKI Jakarta. Saya ing...,Infrastruktur
12859,Kementerian Sosial,BLSM Di Bondowoso,KPS Di desa kemirian kec. tamanan kab.bondowos...,Bantuan Langsung Sementara Masyarakat (BLSM)
12860,Kementerian Sosial,BLSM Tidak Tepat Sasaran di Mataram,KPS BLSM tdk tepat sasaran mRNA ada yg seharus...,Kepesertaan Non-KPS


In [3]:
print('\nNumber of missing values:')
for col in data.columns:
    print('\t%s: %d' % (col, data[col].isnull().sum()))

tipe = data["tipe"].to_list()
data["tipe"].value_counts(dropna=False) #155 kelas diposisi, 37 kelas tipe


Number of missing values:
	diposisi: 141
	judul: 0
	keluhan: 0
	tipe: 0


Bantuan Langsung Sementara Masyarakat (BLSM)                                3592
Kepesertaan Non-KPS                                                         2526
Implementasi Penyesuaian Harga BBM                                           963
Topik Lainnya                                                                806
Beras Miskin (Raskin)                                                        694
Infrastruktur                                                                693
Reformasi Birokrasi dan Tata Kelola                                          643
Kelangkaan BBM                                                               516
Pendidikan                                                                   487
Lingkungan Hidup dan Penanggulangan Bencana                                  413
Kesehatan                                                                    401
Kepesertaan KPS                                                              353
Banjir Jakarta 2013         

In [7]:
# Prepocessing

def tokenize(teks):
    sentence = nlp(teks)
    token_kata = [token.text for token in sentence]
    
    return " ".join(token_kata)
#     return token_kata

def lemma(teks):
    sentence = nlp(teks)

    lemma = [token.lemma_ for token in sentence]
    
    return lemma

def stopword(teks):
    clean = []
    for i in teks:
        if i not in stopwords:
            clean.append(i)

    return clean

In [8]:
def preprocessing(text):
    clean = text.lower() #Lower
    clean = tokenize(clean) #Token
    #Slang
    clean = lemma(clean) #Lemma
    clean = stopword(clean) #Stopword
    clean = ' '.join(ch for ch in clean if ch not in exclude) #Punctuation
    clean = re.sub("[.,]", "", str(clean))
    
    return clean

In [9]:
clean = data["keluhan"].apply(preprocessing)

In [10]:
clean

0        lampu lintas jalan juanda veteran stasiun juan...
1        yth pemprov dki jakarta guru bantu sd harap ma...
2        kps camat patumbak,,kab deli serdang sumut des...
3        tolong angkut mikrolet 39 jurus pademangan man...
4                                          bbm tuju rakyat
5        kps daerah desa sambong blsm .. kadang orang-o...
6        lapor staf izin bangun kec makasar walikota ja...
7        yth perintah provinsi dki jakarta lapor kel ci...
8        kps desa kemirian kec tamanan kab.bondowoso-ja...
9        kps blsm tdk sasar mrna yg tdk link gatep ampe...
10       kps 888bdt 56789009 desa kampung terima 3,5 kg...
11       bbm gmn lapor adu kena spbu yg dijalan wibawa ...
12       kps rumah tangga nama h.abdul manaf jl.hm said...
13       kps diwilayah rw 06 kalideres yg pnya mobil bl...
14       kps tolong tindaklanjuti desa takpernah d data...
15       yth pemprov dki jakarta alam desember 2012 dim...
16       tolong ditinjau pasar tanah abang jalan parkir.

In [17]:
# df = pd.DataFrame(data={"clean": clean})
# df.to_csv("C:/xampp/htdocs/Deep-Learning/clean.csv", sep=',',index=False)

In [2]:
clean_data = pd.read_csv("C:/xampp/htdocs/Deep-Learning/clean.csv",header=0)
clean_data.tail()

,clean
12855,kps daerah desa sambong blsm kadang orang-ora...
12856,lapor staf izin bangun kec makasar walikota ja...
12857,yth perintah provinsi dki jakarta lapor kel ci...
12858,kps desa kemirian kec tamanan kabbondowoso-jat...
12859,kps blsm tdk sasar mrna yg tdk link gatep ampe...


In [9]:
clean = clean_data.clean.values.tolist()
tampung = []
for i in clean:
    sentence = nlp(i)
    token_kata = [token.text for token in sentence]
    tampung.append(token_kata)
    

In [16]:
dictionary = corpora.Dictionary(tampung)
corpus = [dictionary.doc2bow(text) for text in tampung]

pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [19]:
NUM_TOPICS = 10

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics = NUM_TOPICS, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model.save('model5.gensim')

In [27]:
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.024*"rp" + 0.017*"sekolah" + 0.015*"uang" + 0.013*"surat" + 0.012*"daftar"')
(1, '0.151*"yg" + 0.058*"tdk" + 0.039*"dpt" + 0.025*"d" + 0.024*" "')
(2, '0.114*"kps" + 0.095*"blsm" + 0.043*"terima" + 0.038*"desa" + 0.028*"sasar"')
(3, '0.015*"pt" + 0.015*"kendara" + 0.012*"atur" + 0.010*"indonesia" + 0.010*"kota"')
(4, '0.069*"rt" + 0.043*"rw" + 0.031*"nama" + 0.019*"2" + 0.018*"bantu"')
(5, '0.050*"sy" + 0.019*"blm" + 0.018*"ktp" + 0.012*"merah" + 0.010*"jln"')
(6, '0.062*"kartu" + 0.039*"anak" + 0.026*"ulang" + 0.025*"kompensasi" + 0.022*"asi"')
(7, '0.053*"jakarta" + 0.047*"mohon" + 0.032*"kasih" + 0.031*"jalan" + 0.030*"dki"')
(8, '0.027*"air" + 0.025*"ambil" + 0.023*"dagang" + 0.019*"jual" + 0.018*"sampah"')
(9, '0.118*"bbm" + 0.029*"spbu" + 0.023*"harga" + 0.020*"subsidi" + 0.018*"bantu"')


In [ ]:
lemma = list()
for i in list:

In [22]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

In [23]:
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\USER\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [8]:
def tokenize(teks):
    sentence = nlp(teks)
    token_kata = [token.text for token in sentence]
    
#     return "_".join(token_kata)
#     return token_kata

In [10]:
text = "aku belajar komputer"
tes = tokenize(text)
# type(tes

str